Connected to Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset





# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()

projection_dim =128
model = CotrastiveModel(ast_encoder, projection_dim = projection_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

AttributeError: 'ASTEncoder' object has no attribute 'set_train_mode'

Connected to Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset


# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()

projection_dim =128
model = CotrastiveModel(ast_encoder, projection_dim = projection_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

TypeError: CotrastiveModel.__init__() got an unexpected keyword argument 'projection_dim'

In [3]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()

projection_dim =128
model = CotrastiveModel(ast_encoder, projection_dim = projection_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

TypeError: CotrastiveModel.__init__() got an unexpected keyword argument 'projection_dim'

Connected to Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset


# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()

projection_dim =128
model = CotrastiveModel(ast_encoder, projection_dim = projection_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

TypeError: CotrastiveModel.__init__() got an unexpected keyword argument 'projection_dim'

In [3]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

In [4]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

Connected to Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset


# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

Restarted Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset


# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

In [3]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

In [4]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

TypeError: CotrastiveModel.__init__() takes 1 positional argument but 2 were given

Connected to Python 3.10.13

In [1]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------
# dataloaders
# ------------
from data import create_contrastive_datasets, ContrastiveDataset


# 데이터 경로 설정
dataset_dir = "/home/elicer/project/collect_data/mp3"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

#Stage 1 학습 파라미터
num_epochs = 3
batch_size = 256

target_column = 'Set Index'
sim_set_dir = '/home/elicer/project/data/final_final.csv'

# ContrastiveDataset으로 변환
sample_rate = 44100 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, sim_set_dir, target_column, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=batch_size, shuffle=True, drop_last =True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=batch_size, shuffle=False, drop_last= True)
# -> 한 배치의 구성 : clip_a, clip_b, file_id

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------
# model
# ------------
from models import CotrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity




# 1. 모델과 옵티마이저 초기화
######## 여기 만들어야 함!!!!!!!!!!!!
ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()


model = CotrastiveModel(ast_encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [3]:
# 2. BERT 모델 로드 (가사 임베딩용)
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


#3. 학습 모델 로드
def train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = soft_info_nce_loss(
                features=audio_embeddings,
                sim_ij=sim_ij,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
        
    #모델 저장
    torch.save(model, 'WNS_model.pth')
        

def train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            clip_a, clip_b, file_ids, target_value = batch  # 오디오와 file_ids 로드
            clip_a, clip_b = clip_a.to(device), clip_b.to(device)

            # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
            projected_a, projected_b = model(clip_a, clip_b)

            audio_embeddings = torch.cat([projected_a, projected_b], dim=0)  # Combine along the batch dimension
            print(audio_embeddings.shape)

            # 2) 가사 임베딩 생성
            lyrics_embeddings = generate_lyrics_embeddings(file_ids, bert_model, tokenizer, device)

            # 3) 가사 임베딩들 간의 유사도 계산
            sim_ij = compute_similarity(lyrics_embeddings.repeat(2, 1))        

            # 4) 손실 계산
            loss = info_nce_loss(
                features=audio_embeddings,
                batch_size=batch_size,
                n_views=2,
                temperature=0.5,
                device=device
            )
            loss = loss.requires_grad_()

            # 5) 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    #모델 저장
    torch.save(model, 'NS_model.pth')

In [4]:
# ------------
# 학습 - stage 1
# ------------



#둘 중 하나는 빼고 돌리기 
weighting = True
if weighting:
    train_weighted_negative_sampling(train_loader, model, optimizer, bert_model,tokenizer,device,num_epochs,batch_size)
    #staage 2함수
else:
    train_pure_negative_sampling(train_loader,model,optimizer, bert_model,tokenizer, device, num_epochs,batch_size)

Error processing index 389: empty range for randrange() (0, -525939, -525939)


TypeError: CotrastiveModel.forward() missing 1 required positional argument: 'file_id'